# Problema 23 - Application Incident Prediction

## Pré-processamento

### Carregando os dados e realizando limpeza inicial. ###

In [1]:
# Importando bibliotecas necessárias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carregando o dataset
df = pd.read_csv('app_incident_report.csv')

# Removendo a coluna 'incident_duration' conforme especificado no enunciado
df = df.drop('incident_duration', axis=1)

print("Dataset carregado. Dimensões:", df.shape)
df.head()

Dataset carregado. Dimensões: (54745, 8)


,app_name,response_time,error_rate,cpu_usage,memory_usage,disk_space,active_users,downtime
0,NIP,22,61,32,42,52,12000,1
1,NIP,21,63,31,41,51,11000,0
2,NIP,20,98,30,40,50,10000,0
3,NIP,19,15,29,39,49,9000,0
4,NIP,18,67,28,38,48,8000,0


### Analisar se é necessário remover a coluna `app_name`

Olhando os dados por cima notei que o app_name é sempre o mesmo, então nesse passo vamos verificar quantos valores únicos existem na coluna `app_name`. Se houver apenas um, a coluna é constante e não contribui com informação para o modelo, podendo ser removida com segurança.

In [2]:
# Verificando os valores únicos em 'app_name'
unique_apps = df['app_name'].nunique()
app_counts = df['app_name'].value_counts()

print(f"Número de valores únicos em 'app_name': {unique_apps}")
print("\nContagem de cada valor:")
print(app_counts)

if unique_apps == 1:
    print("\nConclusão: A coluna 'app_name' possui um valor constante e será removida.")

Número de valores únicos em 'app_name': 1

Contagem de cada valor:
app_name
NIP    54745
Name: count, dtype: int64

Conclusão: A coluna 'app_name' possui um valor constante e será removida.


### Separação de Características (X) e Alvo (y)

Separando o dataset em Características (features) e Alvo (target):
* **X**: Todas as colunas de características que usaremos para treinar o modelo.
* **y**: A coluna alvo que queremos prever (`downtime`).

In [3]:
# Separando as features (X) e o target (y)
X = df.drop(['downtime', 'app_name'], axis=1)
y = df['downtime']

print("Shape de X (features):", X.shape)
print("Shape de y (target):", y.shape)

Shape de X (features): (54745, 6)
Shape de y (target): (54745,)


### Análise da Variável Alvo (`downtime`)

Pela análise visual foi possível ver que a distribuição da variável `downtime` é completamente
desbalanceada. Esse desbalanceamento pode ser preocupante na hora da divisão dos dados.

In [6]:
# Calculando a contagem e a proporção de cada classe
downtime_counts = y.value_counts()
downtime_percentage = y.value_counts(normalize=True) * 100

print("Contagem de cada classe:")
print(downtime_counts)
print("\nPorcentagem de cada classe:")
print(downtime_percentage)

Contagem de cada classe:
downtime
0    54197
1      548
Name: count, dtype: int64

Porcentagem de cada classe:
downtime
0    98.998995
1     1.001005
Name: proportion, dtype: float64


### Divisão em Dados de Treino e Teste

Será separado 20% dos dados para teste.

Como **demonstrado na análise acima**, a classe `downtime=1` é bem rara (correspondendo a aproximadamente **1%** dos dados). Por isso, é necessário usar o parâmetro `stratify=y`. Essa estratificação garante que a proporção de downtime seja mantida tanto para o conjunto de treino quanto para o de teste.

In [4]:
# Dividindo os dados com test_size=0.2 (20%) e estratificação
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("--- Shapes após a divisão ---")
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

print("\n--- Verificação da estratificação ---")
print("Proporção de downtime no y_train:", y_train.value_counts(normalize=True).values[1])
print("Proporção de downtime no y_test:", y_test.value_counts(normalize=True).values[1])

--- Shapes após a divisão ---
X_train: (43796, 6)
X_test: (10949, 6)
y_train: (43796,)
y_test: (10949,)

--- Verificação da estratificação ---
Proporção de downtime no y_train: 0.010000913325417846
Proporção de downtime no y_test: 0.010046579596310166


### Escalonamento das Características

As colunas numéricas têm escalas muito diferentes. Por exemplo, active_users vai de 0 a 12.000, enquanto cpu_usage vai de 20 a 32.
Portanto, será utilizado o `StandardScaler` para padronizar os dados (média 0 e desvio padrão 1).

**Importante:** O `StandardScaler` é "treinado" (`fit`) **apenas** com os dados de treino para evitar vazamento de informação dos dados de teste. Depois, ele é usado para transformar (`transform`) ambos os conjuntos.

In [5]:
# Criando o objeto scaler
scaler = StandardScaler()

# Treinando o scaler com os dados de treino e transformando-os
X_train_scaled = scaler.fit_transform(X_train)

# Aplicando a mesma transformação aos dados de teste
X_test_scaled = scaler.transform(X_test)

# Convertendo de volta para DataFrame para visualização (opcional)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

print("Amostra dos dados de treino após o escalonamento:")
X_train_scaled_df.head()

Amostra dos dados de treino após o escalonamento:


,response_time,error_rate,cpu_usage,memory_usage,disk_space,active_users
0,0.574671,-0.578807,0.574671,0.574671,0.574671,0.574671
1,1.147714,0.596261,1.147714,1.147714,1.147714,1.147714
2,-1.717498,-1.131780,-1.717498,-1.717498,-1.717498,-1.717498
3,0.001629,-0.820733,0.001629,0.001629,0.001629,0.001629
4,0.001629,-1.028098,0.001629,0.001629,0.001629,0.001629


### Conclusão do Pré-processamento

Os seguintes conjuntos de dados estão preparados:

* `X_train_scaled`, `y_train`: Para treinar os modelos.
* `X_test_scaled`, `y_test`: Para testar os modelos e gerar os resultados finais.